In [1]:
import pandas as pd
import numpy as np 
import statsmodels.api as sm
import itertools
import datetime
from statsmodels import regression
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.factors import Returns, Latest
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.research import run_pipeline

In [6]:
def make_pipeline():
    pipe = Pipeline()
    
    universe = QTradableStocksUS()
    returns = Returns(window_length=2)
   
    mkt_cap = Latest([Fundamentals.market_cap])
    bp = (1 / Latest([Fundamentals.pb_ratio]) )

    big_caps = mkt_cap.rank(mask=universe).top(1000)
    small_caps = mkt_cap.rank(mask=universe).bottom(1000)    
    high_bps = bp.rank(mask=universe).top(1000)
    low_bps = bp.rank(mask=universe).bottom(1000)
    
    universe = QTradableStocksUS() & (big_caps | small_caps | high_bps | low_bps)
    
    pipe = Pipeline(
    columns = {
            'mkt_cap':mkt_cap,
            'bp':bp,
            'big_cap':big_caps,
            'small_cap':small_caps,
            'high_bp':high_bps,
            'low_bp':low_bps,
            'return':returns
        },
        screen=universe
    )
    return pipe

pipe = make_pipeline()

In [7]:
"""
Assume trading day is 2016-01-01, we denote it by t
Formation Period is from t-12*30 to t-1*30, 11 months
Rolling Window start from t-12*30*4
"""

end_date_time = datetime.date(2016, 1, 1)
start_date_time = end_date_time - datetime.timedelta(days = 360*4)

end_date = end_date_time.isoformat()
start_date = start_date_time.isoformat()

results = run_pipeline(pipe, start_date=start_date, end_date=end_date)

In [8]:
results

big_cap        bp high_bp  \
2012-01-23 00:00:00+00:00 Equity(2 [HWM])         True  1.497679    True   
                          Equity(24 [AAPL])       True  0.238498   False   
                          Equity(39 [DDC])       False  0.871764    True   
                          Equity(41 [ARCB])      False  0.950480    True   
                          Equity(52 [ABM])       False  0.723380    True   
                          Equity(53 [ABMD])      False  0.163599   False   
                          Equity(62 [ABT])        True  0.276801   False   
                          Equity(64 [GOLD])       True  0.516102    True   
                          Equity(67 [ADSK])       True  0.265703   False   
                          Equity(76 [TAP])        True  0.975800    True   
                          Equity(88 [ACI])        True  1.164958    True   
                          Equity(99 [ACO])       False  0.458695   False   
                          Equity(110 [RAMP])     False  0.611883    True   
                          Equity(114 [ADBE])      True  0.416198   False   
                          Equity(122 [ADI])       True  0.355999   False   
                          Equity(128 [ADM])       True  0.949487    True   
                          Equity(154 [AEM])       True  0.516102    True   
                          Equity(161 [AEP])       True  0.734322    True   
                          Equity(166 [AES])       True  0.656297    True   
                          Equity(168 [AET])       True  0.685918    True   
                          Equity(185 [AFL])       True  0.641807    True   
                          Equity(197 [AGCO])      True  0.717206    True   
                          Equity(205 [AGN])       True  0.198200   False   
                          Equity(209 [AM])       False  1.600000    True   
                          Equity(216 [HES])       True  0.958865    True   
                          Equity(239 [AIG])       True  2.307337    True   
                          Equity(247 [AIN])      False  0.574713    True   
                          Equity(253 [AIR])      False  1.118819    True   
                          Equity(266 [AJG])       True  0.324202   False   
                          Equity(270 [AKRX])     False  0.149900   False   
...                                                ...       ...     ...   
2016-01-04 00:00:00+00:00 Equity(48126 [HABT])   False  0.123054   False   
                          Equity(48129 [UBS])     True  0.752618    True   
                          Equity(48169 [KLXI])   False  1.374219    True   
                          Equity(48215 [QSR])     True  0.083254   False   
                          Equity(48220 [LC])      True  0.243244   False   
                          Equity(48252 [AVOL])   False  0.639044    True   
                          Equity(48253 [NEWR])   False  0.114456   False   
                          Equity(48257 [HDP])    False  0.099650   False   
                          Equity(48290 [ONDK])   False  0.448997    True   
                          Equity(48304 [BLCM])   False  0.309218   False   
                          Equity(48317 [JUNO])    True  0.259240   False   
                          Equity(48384 [QRVO])    True  0.793980    True   
                          Equity(48390 [UE])     False  0.191316   False   
                          Equity(48486 [BOX])    False  0.099730   False   
                          Equity(48487 [EURN])   False  0.824363    True   
                          Equity(48531 [VSTO])   False  0.600522    True   
                          Equity(48543 [SHAK])   False  0.091514   False   
                          Equity(48544 [HIFR])   False  0.828972    True   
                          Equity(48547 [ONCE])   False  0.171201   False   
                          Equity(48575 [XHR])    False  0.986715    True   
                          Equity(48629 [INOV])   False  0.279497   False   
                   

In [13]:
"""
Rf: US treasury bill ETF
Rm: S&P 500 ETF
"""

rf = get_pricing('BIL', fields='price', start_date=start_date, end_date=end_date).pct_change()[1:]
rm = get_pricing('SPY', fields='price', start_date=start_date, end_date=end_date).pct_change()[1:]

Rm_Rf = rm - rf

SMB = results[results.small_cap]['return'].groupby(level=0).mean() - \
        results[results.big_cap]['return'].groupby(level=0).mean()
        
HML = results[results.high_bp]['return'].groupby(level=0).mean() - \
        results[results.low_bp]['return'].groupby(level=0).mean()

data = results[['return']].set_index(results.index)
assets_num = [group[1].size for group in data.groupby(level=0)] #group by date

Rm_Rf_col = [[Rm_Rf.loc[group[0]]]*size if group[0] in Rm_Rf.index else [None]*size \
             for group, size in zip(data.groupby(level=0), assets_num)]
SMB_col = [[SMB.loc[group[0]]] * size for group, size in zip(data.groupby(level=0), assets_num)]
HML_col = [[HML.loc[group[0]]] * size for group, size in zip(data.groupby(level=0), assets_num)]

data['Rm_Rf'] = list(itertools.chain(*Rm_Rf_col))
data['SMB'] = list(itertools.chain(*SMB_col))
data['HML'] = list(itertools.chain(*HML_col))
data = data.dropna()

In [14]:
data

return     Rm_Rf       SMB  \
2012-01-24 00:00:00+00:00 Equity(2 [HWM])       0.007375 -0.001064 -0.001749   
                          Equity(24 [AAPL])     0.016894 -0.001064 -0.001749   
                          Equity(39 [DDC])      0.032907 -0.001064 -0.001749   
                          Equity(41 [ARCB])    -0.009095 -0.001064 -0.001749   
                          Equity(52 [ABM])      0.024586 -0.001064 -0.001749   
                          Equity(53 [ABMD])    -0.042988 -0.001064 -0.001749   
                          Equity(62 [ABT])     -0.000717 -0.001064 -0.001749   
                          Equity(64 [GOLD])     0.024113 -0.001064 -0.001749   
                          Equity(67 [ADSK])    -0.011130 -0.001064 -0.001749   
                          Equity(76 [TAP])     -0.000227 -0.001064 -0.001749   
                          Equity(88 [ACI])      0.018012 -0.001064 -0.001749   
                          Equity(99 [ACO])     -0.007204 -0.001064 -0.001749   
                          Equity(110 [RAMP])   -0.005061 -0.001064 -0.001749   
                          Equity(114 [ADBE])   -0.008852 -0.001064 -0.001749   
                          Equity(122 [ADI])    -0.006791 -0.001064 -0.001749   
                          Equity(128 [ADM])    -0.005657 -0.001064 -0.001749   
                          Equity(154 [AEM])     0.009966 -0.001064 -0.001749   
                          Equity(161 [AEP])    -0.004880 -0.001064 -0.001749   
                          Equity(166 [AES])    -0.007663 -0.001064 -0.001749   
                          Equity(168 [AET])    -0.003201 -0.001064 -0.001749   
                          Equity(185 [AFL])     0.002923 -0.001064 -0.001749   
                          Equity(197 [AGCO])    0.019200 -0.001064 -0.001749   
                          Equity(205 [AGN])     0.018575 -0.001064 -0.001749   
                          Equity(209 [AM])      0.004228 -0.001064 -0.001749   
                          Equity(216 [HES])     0.006530 -0.001064 -0.001749   
                          Equity(239 [AIG])    -0.008774 -0.001064 -0.001749   
                          Equity(247 [AIN])    -0.024020 -0.001064 -0.001749   
                          Equity(253 [AIR])    -0.005636 -0.001064 -0.001749   
                          Equity(266 [AJG])     0.008719 -0.001064 -0.001749   
                          Equity(270 [AKRX])    0.001832 -0.001064 -0.001749   
...                                                  ...       ...       ...   
2015-12-31 00:00:00+00:00 Equity(48104 [PGRE])  0.000000 -0.009712 -0.003549   
                          Equity(48126 [HABT]) -0.023970 -0.009712 -0.003549   
                          Equity(48129 [UBS])  -0.010045 -0.009712 -0.003549   
                          Equity(48169 [KLXI]) -0.014035 -0.009712 -0.003549   
                          Equity(48215 [QSR])  -0.011705 -0.009712 -0.003549   
                          Equity(48220 [LC])   -0.014559 -0.009712 -0.003549   
                          Equity(48252 [AVOL])  0.001297 -0.009712 -0.003549   
                          Equity(48253 [NEWR])  0.004933 -0.009712 -0.003549   
                          Equity(48257 [HDP])  -0.021286 -0.009712 -0.003549   
                          Equity(48290 [ONDK])  0.002910 -0.009712 -0.003549   
                          Equity(48304 [BLCM])  0.005345 -0.009712 -0.003549   
                          Equity(48317 [JUNO]) -0.019060 -0.009712 -0.003549   
                          Equity(48384 [QRVO]) -0.025099 -0.009712 -0.003549   
                          Equity(48390 [UE])   -0.005860 -0.009712 -0.003549   
                          Equity(48486 [BOX])  -0.016289 -0.009712 -0.003549   
                          Equity(48487 [EURN])  0.014169 -0.009712 -0.003549   
                          Equity(48531 [VSTO])  0.001556 -0.009712 -0.003549   
                          Equity(48543 [SHAK]) -0.017462 -0.009712 -0.003549   
                          Equity(48544 [HIFR]) -0.

In [26]:
# Formation Period [t-J, t-1]
month_Cum_RR = pd.Series()
RR_VAR = pd.Series()

for j in range(11):
    # prediction start date is one day after rolling period
    # find the closest date cuz some date we set might not be trading day, otherwise cannot be located later
    roll_end = min(data.index.levels[0].date, \
                   key=lambda x: abs(x-(end_date_time - datetime.timedelta(days = 360-30*j)))).isoformat()
    roll_start = min(data.index.levels[0].date, \
                     key=lambda x: abs(x-(end_date_time - datetime.timedelta(days = 360*4-30*j)))).isoformat()
    pre_end = min(data.index.levels[0].date, \
                  key=lambda x: abs(x-(end_date_time - datetime.timedelta(days = 360-30*(j+1))))).isoformat()
    pre_start = min(data.index.levels[0].date, \
                    key=lambda x: abs(x-(end_date_time - datetime.timedelta(days = 359-30*j)))).isoformat()

    df1 = data.loc[roll_start:roll_end] # Rolling window
    df2 = data.loc[pre_start:pre_end] # Prediction month

    # FF3 Regression *************
    df1 = sm.add_constant(df1)
    assets = df1.index.levels[1].unique()

    Y = [df1.xs(asset, level=1)['return'] for asset in assets]
    X = [df1.xs(asset, level=1)[['const', 'Rm_Rf', 'SMB', 'HML']] for asset in assets]

    reg_results  = [regression.linear_model.OLS(y, x).fit().params for y, x in zip(Y, X) if not(x.empty or y.empty)]
    indices = [asset for y, x, asset in zip(Y, X, assets) if not(x.empty or y.empty)]
    betas = pd.DataFrame(reg_results, index=indices)
    del betas['const']
    betas['return'] = np.ones(len(betas))
    betas = betas.ix[:, df2.columns]

    # Prediction *************
    RR_data = df2.multiply(betas, axis=0, level=1)
    RR_data['return_hat'] = (RR_data['Rm_Rf']+ RR_data['SMB']+ RR_data['HML'])
    RR_data['RR'] = (RR_data['return']- RR_data['return_hat'])

    # Calculate Cumulative Residual Return for this month, which is the cumsum value on the last day
    # Directly use Var() to get the Residual Return Var for this month
    month_Cum_RR = month_Cum_RR.append(RR_data['RR'].groupby(level=1).cumsum(axis=0)[pre_end])
    RR_VAR = RR_VAR.append(RR_data['RR'].groupby(level=1).var())

In [27]:
RR = month_Cum_RR.to_frame(name='CumRR')
CumRR = [[name, group[1].cumsum(axis=0).iloc[-1][0]] for name, group in zip(RR.index.unique(), RR.groupby(level=0))]

VAR = RR_VAR.to_frame(name='VAR')
CumVAR = [[name, group[1].cumsum(axis=0).iloc[-1][0]] for name, group in zip(VAR.index.unique(), VAR.groupby(level=0))]

C1 = pd.DataFrame(CumRR, columns =['Equity', 'CumRR']).set_index('Equity')
C2 = pd.DataFrame(CumVAR, columns =['Equity', 'CumVAR']).set_index('Equity')
Combine = pd.concat([C1, C2], axis=1).dropna()
Combine['sqrtCumVAR'] = np.sqrt(Combine['CumVAR'])
Combine['CumRR/sqrtCumVAR'] = Combine['CumRR']/Combine['sqrtCumVAR'] 

In [33]:
Ranks = (Combine['CumRR']/Combine['sqrtCumVAR']).sort_values(ascending=False)
num = round(len(Ranks)*0.01)
Portfolio = pd.DataFrame(list(zip(Ranks.head(num).index, Ranks.tail(num).index)), columns=['Long', 'Short'])
print(Portfolio)

                    Long                 Short
0     Equity(6426 [RES])  Equity(23718 [PNFP])
1   Equity(42037 [ZLTQ])  Equity(20374 [HCBK])
2   Equity(40547 [TRGP])  Equity(43984 [CONE])
3   Equity(21447 [SGMO])   Equity(8566 [UEIC])
4    Equity(34300 [OWW])  Equity(47495 [OTIC])
5     Equity(32603 [WU])    Equity(6098 [POM])
6    Equity(7581 [TRMK])    Equity(7401 [TER])
7   Equity(26496 [BLKB])  Equity(26315 [NILE])
8    Equity(11514 [LPT])  Equity(18759 [IOSP])
9   Equity(11512 [LJPC])  Equity(39860 [QUAD])
10   Equity(22406 [UPL])  Equity(14255 [IDRA])
11    Equity(4914 [MMC])  Equity(25319 [DTSI])
12  Equity(25320 [ECPG])  Equity(14596 [ELNK])
13  Equity(16945 [RMBS])  Equity(43494 [SSTK])
14   Equity(38691 [CFN])   Equity(45499 [FMI])
15   Equity(18834 [AVB])  Equity(10417 [ARWR])
16   Equity(6163 [PRGS])  Equity(23267 [WOOF])
17  Equity(12553 [ISSI])   Equity(7158 [STKL])
18    Equity(2470 [EGN])   Equity(13601 [THG])
19   Equity(18584 [LNT])   Equity(22110 [DVA])
20   Equity(2

In [34]:
Ranks

Equity(6426 [RES])      166.862646
Equity(42037 [ZLTQ])    144.588363
Equity(40547 [TRGP])    122.368189
Equity(21447 [SGMO])    110.483881
Equity(34300 [OWW])      81.565457
Equity(32603 [WU])       74.171287
Equity(7581 [TRMK])      69.476735
Equity(26496 [BLKB])     51.417922
Equity(11514 [LPT])      40.537628
Equity(11512 [LJPC])     39.233923
Equity(22406 [UPL])      38.827869
Equity(4914 [MMC])       38.456867
Equity(25320 [ECPG])     36.208194
Equity(16945 [RMBS])     35.138092
Equity(38691 [CFN])      34.422081
Equity(18834 [AVB])      34.156997
Equity(6163 [PRGS])      32.091050
Equity(12553 [ISSI])     31.102234
Equity(2470 [EGN])       28.362966
Equity(18584 [LNT])      28.165685
Equity(26969 [EDR])      27.587499
Equity(14329 [ANDE])     27.123744
Equity(8014 [VNO])       26.817600
Equity(6347 [RAVN])      26.470085
Equity(14388 [IRM])      26.318900
Equity(45249 [CVT])      25.908052
Equity(27543 [EXPE])     25.570297
Equity(42950 [FB])       24.963537
Equity(23176 [ABCO])